In [ ]:
import os
import re
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

%matplotlib inline

In [ ]:
def compute_mean(dfs, elem_name, last_year=2022, year_window=[5,30]):
    n = len(dfs)
    recent, history = np.nan + np.zeros(n), np.nan + np.zeros(n)
    for i,df in enumerate(dfs):
        if elem_name in df:
            tmp = df.groupby('year').agg(np.nanmean)[elem_name]
            year = tmp.index.to_numpy()
            elem = tmp.to_numpy()
            idx = (year>last_year-year_window[0]) & (year<=last_year)
            jdx = (year>last_year-np.sum(year_window)) & (year<=last_year-year_window[0])
            if any(idx) and any(jdx):
                recent[i] = elem[idx].mean()
                history[i] = elem[jdx].mean()
    return recent, history

## Map of Europe

In [ ]:
scale = 1 # 1 : 1,000,000
year = 2021
coord_ref = 3035 # coordinate reference system
                 # 3035: Lambert azimuthal equal area
                 # 3857: spherical Mercator projection
                 # 4326: world geodetic system 1984
europe_folder = f'geography/ref-nuts-{year}-{scale:02d}m'
N_levels = 4
map_types = 'BN', 'LB' # BN: boundary, LB: label, RG: region
europe = {map_type: {} for map_type in map_types}
for level in range(N_levels):
    gdf = {}
    for map_type in map_types:
        if map_type == 'LB':
            europe_file = f'{europe_folder}/NUTS_{map_type}_{year}_{coord_ref}_LEVL_{level}.json'
        else:
            europe_file = f'{europe_folder}/NUTS_{map_type}_{scale:02d}M_{year}_{coord_ref}_LEVL_{level}.json'
        gdf = gpd.read_file(europe_file)
        gdf.crs = CRS.from_user_input(coord_ref)
        europe[map_type][level] = gdf

## Bounding box

In [ ]:
bbox = {'WS': Point(-9.5, 30), 'EN': Point(75, 60)}
points = gpd.GeoDataFrame(data=bbox.values(),
                          index=pd.Index(data=bbox.keys(), name='name'),
                          columns=['geometry'])
points.loc['home'] = Point(9.400984, 44.272555)
points.crs = CRS.from_user_input(4326)
points = points.to_crs(epsg=coord_ref)

## Weather stations

In [ ]:
country_codes = 'AJ','AL','AM','AU','BE','BK','BO','BU','CY','DA',\
                'EI','EN','EZ','FI','FR','GG','GM','IT','SW','LG',\
                'LH','LO','LU','MD','MJ','MK','MT','NL'
data_dir = 'ghcnd_all'
dfs, station_IDs = [], []
for code in country_codes:
    data_files = sorted(glob.glob(os.path.join(data_dir, code + '*.parquet*')))
    for f in tqdm(data_files):
        station_IDs.append(os.path.basename(f).split('.')[0])
        dfs.append(pd.read_parquet(f))

In [ ]:
last_year = 2022
window = [5, 30]
data = {}
for elem_name in ('tmax', 'tmin'):
    rec,hist = compute_mean(dfs, elem_name, last_year, window)
    data[elem_name + '_recent'] = rec
    data[elem_name + '_history'] = hist
    data[elem_name + '_delta'] = rec - hist
df = pd.DataFrame(data=data, index=station_IDs)

In [ ]:
stations = pd.read_csv('ghcnd-stations.csv', header=None, index_col=0, \
                       names=['latitude','longitude', 'elevation', \
                              'name', 'prcp', 'prcp_attributes'])
stations = stations.join(df, how='inner')
stations['geometry'] = [Point(long, lat) for lat,long 
                        in stations.loc[:,['latitude','longitude']].itertuples(index=False)]
stations = gpd.GeoDataFrame(stations[['name','tmax_delta','tmin_delta','geometry']])
stations.crs = CRS.from_user_input(4326)
stations = stations.to_crs(epsg=coord_ref)

In [ ]:
XY = np.zeros((len(stations), 2))
for i,point in enumerate(stations.geometry):
    XY[i,0], XY[i,1] = point.x, point.y
ΔT = stations.tmax_delta.to_numpy()
idx, = np.where(np.logical_not(np.isnan(ΔT)))
ΔT = ΔT[idx]
XY = XY[idx]
ΔT[ΔT > 5] = 5
ΔT[ΔT < -5] = -5
interp = LinearNDInterpolator(XY, ΔT)

In [ ]:
bin_size = 10000
x = np.r_[XY[:,0].min() : XY[:,0].max()+bin_size/2 : bin_size]
y = np.r_[XY[:,1].min() : XY[:,1].max()+bin_size/2 : bin_size]
X,Y = np.meshgrid(x, y)
Z = interp(np.array([X.flatten(), Y.flatten()]).T).reshape(X.shape)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(12,8))
light_gray = .75 + np.zeros(3)
dark_gray = .3 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1, color=dark_gray)
# europe['BN'][1].plot(ax=ax, lw=0.75, color=light_gray)
# europe['BN'][2].plot(ax=ax, lw=0.75, color=light_gray)
# europe['BN'][3].plot(ax=ax, lw=0.75, color=light_gray)
ax.pcolormesh(X, Y, Z, vmin=-5, vmax=5, cmap='coolwarm')
ax.set_xlim([points.loc['WS','geometry'].x, points.loc['EN','geometry'].x])
ax.set_ylim([points.loc['WS','geometry'].y, points.loc['EN','geometry'].y])
ax.axis('off')
fig.tight_layout()